# Use GNU Octave in your C++ program

> *Dec 18, 2019 — Kai T. Ohlhus &lt;<k.ohlhus@gmail.com>&gt; — [CC BY 4.0](http://creativecommons.org/licenses/by/4.0/)*

## Using built-in (C++) Octave functions

In case you want to use some Octave function inside **YOUR** C++ program,
you can include Octave code, just like any other programming library.

The easiest way is to include so-called "built-in" functions,
as they are also written in C++.
**No interpretation of m-code** is necessary
and convenient data structures like `Matrix` are available to your program.

To find out, if an Octave function was built-in.  Just call, for example,

In [1]:
which norm

'norm' is a built-in function from the file libinterp/corefcn/data.cc


in contrast to

In [ ]:
which normest

that is treated in the second part of this notebook.

In general, we wan

In [2]:
A = [11, 12; ...
     21, 22]
norm (A)

A =

   11   12
   21   22

ans =  34.495


In [3]:
type functions/standalone_builtin.cc

#include <iostream>
#include <octave/oct.h>
#include <octave/builtin-defun-decls.h>

int main () {
  int n = 2;
  Matrix A = Matrix (n, n);

  // Fill matrix
  for (octave_idx_type i = 0; i < n; i++) {
    for (octave_idx_type j = 0; j < n; j++) {
      A(i,j) = (i + 1) * 10 + (j + 1);
    }
  }

  // Compute matrix norm
  octave_value_list out = Fnorm (ovl (A), 1);

  // Output results
  std::cout << "Matrix A:"            << std::endl
            << A                      << std::endl
            << "norm (A) = "
            << out(0).double_value () << std::endl;

  return 0;
}



In [4]:
!mkoctfile --link-stand-alone functions/standalone_builtin.cc -o standalone_builtin.exe

In [5]:
!ldd ./standalone_builtin.exe

	linux-vdso.so.1 (0x00007ffdfd3df000)
	liboctinterp.so.7 => not found
	liboctave.so.7 => not found
	libstdc++.so.6 => /usr/lib64/libstdc++.so.6 (0x00007fb04514f000)
	libm.so.6 => /lib64/libm.so.6 (0x00007fb044e17000)
	libgomp.so.1 => /usr/lib64/libgomp.so.1 (0x00007fb044be7000)
	libgcc_s.so.1 => /lib64/libgcc_s.so.1 (0x00007fb0449cf000)
	libpthread.so.0 => /lib64/libpthread.so.0 (0x00007fb0447b1000)
	libc.so.6 => /lib64/libc.so.6 (0x00007fb0443f7000)
	/lib64/ld-linux-x86-64.so.2 (0x00007fb0454d9000)
	libdl.so.2 => /lib64/libdl.so.2 (0x00007fb0441f3000)



In [6]:
![[ ":$LD_LIBRARY_PATH:" != *":/home/siko1056/work/octave/GNU-Octave-enable-64/install/lib/octave/5.1.0/:"* ]] && export LD_LIBRARY_PATH="/home/siko1056/work/octave/GNU-Octave-enable-64/install/lib/octave/5.1.0/:${LD_LIBRARY_PATH}"
!echo "$LD_LIBRARY_PATH"

/home/siko1056/work/octave/GNU-Octave-enable-64/install/lib/octave/5.1.0/::/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/lib64



In [7]:
!./standalone_builtin.exe

Matrix A:
 11 12
 21 22

norm (A) = 34.4952



## Using ANY Octave function

In [8]:
type functions/standalone_embedded.cc

#include <iostream>
#include <octave/oct.h>
#include <octave/octave.h>
#include <octave/parse.h>
#include <octave/interpreter.h>

int main (void) {
  int n = 2;
  Matrix A = Matrix (n, n);

  // Fill matrix
  for (octave_idx_type i = 0; i < n; i++) {
    for (octave_idx_type j = 0; j < n; j++) {
      A(i,j) = (i + 1) * 10 + (j + 1);
    }
  }

  octave::interpreter interpreter;
  octave_value_list out;

  try {
    int status = interpreter.execute ();
    if (status != 0) {
      std::cerr << "An error occurred!" << std::endl;
      return status;
    }

    out = octave::feval ("normest", ovl (A), 1);

  } catch (const octave::exit_exception& ex) {
    std::cerr << "Octave interpreter exited with status = "
              << ex.exit_status () << std::endl;
  } catch (const octave::execution_exception&) {
    std::cerr << "An error encountered in Octave evaluator!" << std::endl;
  }

  // Output results
  std::cout << "Matrix A:"            << std::endl
            << A                

In [9]:
!mkoctfile --link-stand-alone functions/standalone_embedded.cc -o standalone_embedded.exe

In [12]:
!./standalone_embedded.exe

Matrix A:
 11 12
 21 22

normest (A) = 34.4952

